Import bibliotekii, wczytanie plików.
Dodanie kolumny stanu.

In [34]:
import pandas as pd
restaurants = pd.read_csv('restaurants.csv')
menu = pd.read_csv('restaurant-menus.csv')
merged = restaurants.merge(menu, how='left', left_on='id', right_on='restaurant_id' , sort=False, suffixes=('_rest', '_menu'))
restaurants['state'] = [str(a).replace(str(b), '').strip()[-3:-1] for a, b in zip(restaurants['full_address'], restaurants['zip_code'])]
restaurants.loc[restaurants['zip_code'].str.len() == 2, ['zip_code']] = '00000'
restaurants.loc[restaurants['zip_code'].isin(['75070']), ['state']] = 'TX'
restaurants.loc[restaurants['state'].isin([', ','an','U']),['state']] = 'PR'
restaurants['state'].unique()
restaurants[restaurants['state']=='']


,id,position,name,score,ratings,category,price_range,full_address,zip_code,lat,lng,state
110,111,124,Post Office Pies (Avondale),4.6,32.0,"Pizza, American, Italian",$,NaN,NaN,33.52467,-86.77390,
129,130,100,Urban Cookhouse (Birmingham),4.8,19.0,"American, Sandwich, Salads",NaN,NaN,NaN,33.51668,-86.81039,
143,144,80,Godfather's Pizza (231 State Farm Pkwy),4.9,16.0,"Pizza, American, Italian",$$,NaN,NaN,33.44629,-86.82708,
584,585,35,It's Just Wings (3083 Montgomery Highway),3.7,31.0,"American, Wings",$$,NaN,NaN,31.25078,-85.42848,
899,900,1,Five Guys AL- 4062 5413 US 280,NaN,NaN,"American, Burgers",$,NaN,NaN,33.24693,-86.33891,
...,...,...,...,...,...,...,...,...,...,...,...,...
39274,39275,183,Memo’s tacos,NaN,NaN,"Mexican, Latin American, New Mexican",$$,NaN,NaN,33.00313,-97.22769,
39303,39304,36,"Qdoba (1521 Keller Parkway, Suite 700)",NaN,NaN,"Dinner, Mexican, Burritos, Tacos",$,NaN,NaN,32.93468,-97.21780,
39465,39466,98,Asian Kitchen,NaN,NaN,"Chinese, Asian, Asian Fusion",NaN,NaN,NaN,30.33749,-97.69273,
39629,39630,280,Fish &amp; Rice Poke Bar,NaN,NaN,"Asian, Asian Fusion, Korean",$,NaN,NaN,30.33537,-97.72005,


Wyodrębnienie danych dla poszczególnych kategorii cenowych.

In [36]:
cheap = restaurants[restaurants['price_range']=='$']
moderate = restaurants[restaurants['price_range']=='$$']
expensive = restaurants[restaurants['price_range']=='$$$']
superexpensive = restaurants[restaurants['price_range']=='$$$$']

Ile jest tanich restauracji w poszczególny stanach?

In [37]:
cheapsy = cheap.groupby('state', as_index=False)['price_range'].count()
#zmiana nazwy na unikatową
cheapsy.rename(columns = {'price_range': 'cheap #'}, inplace=True)
cheapsy

,state,cheap #
0,,94
1,AL,643
2,DC,778
3,ID,15
4,IL,141
5,MD,528
6,MN,23
7,OH,5
8,OR,583
9,PR,136


Ile jest restauracji o umiarkowanych cenach?

In [38]:
moderasy = moderate.groupby('state', as_index=False)['price_range'].count()
#zmiana nazwy na unikatową
moderasy.rename(columns = {'price_range': 'moderate #'}, inplace=True)
moderasy

,state,moderate #
0,,76
1,AL,300
2,DC,297
3,ID,8
4,IL,43
5,MD,177
6,MN,16
7,OH,10
8,OR,122
9,PR,13


Ile jest drogich restauracji?

In [39]:
expensy = expensive.groupby('state', as_index=False)['price_range'].count()
#zmiana nazwy na unikatową
expensy.rename(columns = {'price_range': 'expensive #'}, inplace=True)
expensy

,state,expensive #
0,,2
1,AL,6
2,DC,23
3,MD,7
4,OR,3
5,TX,18
6,UT,1
7,VA,40
8,WA,41
9,WI,6


Ile jest BARDZO drogich restauracji?

In [40]:
superesy = superexpensive.groupby('state', as_index=False)['price_range'].count()
superesy.rename(columns = {'price_range': 'superexpensive #'}, inplace=True)
#zmiana nazwy na unikatową
superesy

,state,superexpensive #
0,DC,2
1,TX,2
2,UT,1
3,VA,5
4,WA,7
5,WI,1


Połączenie wszystkich kategorii w tabelę całościową i wyliczenie wartości całkowitej (tj. ile jest restauracji z zakresem cen w danym stanie)

In [41]:
#nie ma mozliwosci łączenia wielu dataframeów na raz, więc robię to stopniowo
chm = cheapsy.merge(moderasy, how='outer', on ='state')
exmch = chm.merge(expensy, how='outer', on ='state')
all = exmch.merge(superesy, how='outer', on ='state')
#usuniecie wartosci NaN
all = all.fillna(0)
#nowa kolumna total
all['total'] = all['cheap #'] + all['moderate #'] + all['expensive #'] + all['superexpensive #']
all

,state,cheap #,moderate #,expensive #,superexpensive #,total
0,,94,76,2.0,0.0,172.0
1,AL,643,300,6.0,0.0,949.0
2,DC,778,297,23.0,2.0,1100.0
3,ID,15,8,0.0,0.0,23.0
4,IL,141,43,0.0,0.0,184.0
5,MD,528,177,7.0,0.0,712.0
6,MN,23,16,0.0,0.0,39.0
7,OH,5,10,0.0,0.0,15.0
8,OR,583,122,3.0,0.0,708.0
9,PR,136,13,0.0,0.0,149.0


Ile procentowo stanowią określone kategorie cenowe w konkretnych stanach?

In [42]:
all['cheap #'] = all['cheap #'] / all['total'] * 100
all.rename(columns = {'cheap #': 'cheap %'}, inplace=True)
all['moderate #'] = all['moderate #'] / all['total'] * 100
all.rename(columns = {'moderate #': 'moderate %'}, inplace=True)
all['expensive #'] = all['expensive #'] / all['total'] * 100
all.rename(columns = {'expensive #': 'expensive %'}, inplace=True)
all['superexpensive #'] = all['superexpensive #'] / all['total'] * 100
all.rename(columns = {'superexpensive #': 'superexpensive %'}, inplace=True)

In [43]:
#zaokrąglenie do dwóch miejsc po przecinku
all.round(2)

,state,cheap %,moderate %,expensive %,superexpensive %,total
0,,54.65,44.19,1.16,0.00,172.0
1,AL,67.76,31.61,0.63,0.00,949.0
2,DC,70.73,27.00,2.09,0.18,1100.0
3,ID,65.22,34.78,0.00,0.00,23.0
4,IL,76.63,23.37,0.00,0.00,184.0
5,MD,74.16,24.86,0.98,0.00,712.0
6,MN,58.97,41.03,0.00,0.00,39.0
7,OH,33.33,66.67,0.00,0.00,15.0
8,OR,82.34,17.23,0.42,0.00,708.0
9,PR,91.28,8.72,0.00,0.00,149.0


Co nam dają te informacje?  
Mozna załozyc, ze otworzenie drogiej restauracji w stanie, w ktorym jest znaczna przewaga restauracji tanich   
(np. VA: na prawie 8k restauracji ponad 80% stanowią tanie restauracje) nie miałoby większego sensu.  


Lepszym pomysłem byłby stan, w którym mamy do czynienia z nizszym stosunkiem restauracji o niskim  
zakresie cenowym i pojedyncze drogie restauracje: np. WI, WV, VT.   

Niestety rzuca się tu w oczy niedoskonałość naszych danych, tj. w niektórych stanach mamy tysiące restauracji, a w innych kilka/kilkadziesiąt.  


Co warto poprawić?  
Moze scalić kategorie superexpensive i expensive, bo danych jest bardzo mało.
Az 172 wyniki nie mają przypisanego stanu, rozwazyć dodanie na podstawie długości i szerokości.




Ile procentowo stanowią poszczególne kategorie cenowe (bez podziału na stany)?

In [62]:
df = restaurants.value_counts('price_range').rename_axis('price_range').to_frame('total')
df['sum'] = df['total'].sum()
df['sum'] = df['total'] / df['sum'] * 100
df = df.rename(columns={'sum': '%'})
df = df.round(2)
df



,total,%
price_range,,
$,24385,72.62
$$,9029,26.89
$$$,149,0.44
$$$$,18,0.05
